In [1]:
import pandas as pd

# Non-neural models

In [6]:
data = pd.read_csv("./data/splited_RASFF_DATA_16092019.csv", sep=";", header=0, index_col=0)

data.head(3)

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
0,2019-09-13,France,meat and meat products (other than poultry),food,serious,no stock left,no distribution from notifying country,pathogenic micro-organisms,Romania,France,Belgium
1,2019-09-13,Denmark,"cocoa and cocoa preparations, coffee and tea",food,serious,recall from consumers,no distribution from notifying country,natural toxins (other),Germany,Denmark,
2,2019-09-13,Poland,confectionery,food,serious,informing authorities,distribution restricted to notifying country,food additives and flavourings,Croatia,Poland,
